In [1]:
import dspy
import ujson
import random

gpt4o = dspy.LM("openai/gpt-4o", temperature=0.7)
dspy.configure(lm=gpt4o)

In [2]:
from dspy.utils import download

download(
    "https://huggingface.co/datasets/bytedance-research/ToolHop/resolve/main/data/ToolHop.json"
)

data = ujson.load(open("ToolHop.json"))
random.Random(0).shuffle(data)

In [3]:
import re
import inspect

examples = []
fns2code = {}


def finish(answer: str):
    """Conclude the trajectory and return the final answer."""
    return answer


for datapoint in data:
    func_dict = {}
    for func_code in datapoint["functions"]:
        cleaned_code = func_code.rsplit("\n\n# Example usage", 1)[0]
        fn_name = re.search(r"^\s*def\s+([a-zA-Z0-9_]+)\s*\(", cleaned_code)
        fn_name = fn_name.group(1) if fn_name else None

        if not fn_name:
            continue

        local_vars = {}
        exec(cleaned_code, {}, local_vars)
        fn_obj = local_vars.get(fn_name)

        if callable(fn_obj):
            func_dict[fn_name] = fn_obj
            assert fn_obj not in fns2code, f"Duplicate function found: {fn_name}"
            fns2code[fn_obj] = (fn_name, cleaned_code)

    func_dict["finish"] = finish

    example = dspy.Example(
        question=datapoint["question"], answer=datapoint["answer"], functions=func_dict
    )
    examples.append(example.with_inputs("question", "functions"))

trainset, devset, testset = examples[:100], examples[100:400], examples[400:]

<string>:47: SyntaxWarning: invalid escape sequence '\d'


In [6]:
from func_timeout import func_set_timeout


def wrap_function_with_timeout(fn):
    @func_set_timeout(10)
    def wrapper(*args, **kwargs):
        try:
            return {"return_value": fn(*args, **kwargs), "errors": None}
        except Exception as e:
            return {"return_value": None, "errors": str(e)}

    return wrapper


def fn_metadata(func):
    signature = inspect.signature(func)
    docstring = inspect.getdoc(func) or "No docstring."
    return dict(
        function_name=func.__name__, arguments=str(signature), docstring=docstring
    )


def metric(example, pred, trace=None):
    gold = str(example.answer).rstrip(".0").replace(",", "").lower()
    pred = str(pred.answer).rstrip(".0").replace(",", "").lower()
    return pred == gold  # stricter than the original paper's metric!


evaluate = dspy.Evaluate(
    devset=devset,
    metric=metric,
    num_threads=24,
    display_progress=True,
    display_table=0,
    max_errors=999,
)

In [7]:
class Agent(dspy.Module):
    def __init__(self, max_steps=5):
        self.max_steps = max_steps
        instructions = "For the final answer, produce short (not full sentence) answers in which you format dates as YYYY-MM-DD, names as Firstname Lastname, and numbers without leading 0s."
        signature = dspy.Signature(
            "question, trajectory, functions -> next_selected_fn, args: dict[str, Any]",
            instructions,
        )
        self.react = dspy.ChainOfThought(signature)

    def forward(self, question, functions):
        tools = {fn_name: fn_metadata(fn) for fn_name, fn in functions.items()}
        trajectory = []

        for _ in range(self.max_steps):
            pred = self.react(question=question, trajectory=trajectory, functions=tools)
            selected_fn = pred.next_selected_fn.strip('"').strip("'")
            fn_output = wrap_function_with_timeout(functions[selected_fn])(**pred.args)
            trajectory.append(
                dict(
                    reasoning=pred.reasoning,
                    selected_fn=selected_fn,
                    args=pred.args,
                    **fn_output,
                )
            )

            if selected_fn == "finish":
                break

        return dspy.Prediction(
            answer=fn_output.get("return_value", ""), trajectory=trajectory
        )

In [8]:
agent = Agent()
evaluate(agent)

Average Metric: 100.00 / 267 (37.5%):  89%|████████▉ | 267/300 [02:21<00:21,  1.53it/s]

2025/07/20 13:04:27 ERROR dspy.utils.parallelizer: Error for Example({'question': 'What is the date 10 weeks after the date of death of the director of film Cross Current?', 'answer': '2014-05-18', 'functions': {'film_info_retriever': <function film_info_retriever at 0x13e07a0c0>, 'sports_figure_bio_retriever': <function sports_figure_bio_retriever at 0x13e07a520>, 'date_calculator': <function date_calculator at 0x13e07a5c0>, 'finish': <function finish at 0x13d2ea520>}}) (input_keys={'functions', 'question'}): 'general_bio_retriever'. Set `provide_traceback=True` for traceback.


Average Metric: 109.00 / 299 (36.5%): 100%|██████████| 300/300 [02:38<00:00,  1.89it/s]

2025/07/20 13:04:43 INFO dspy.evaluate.evaluate: Average Metric: 109.0 / 300 (36.3%)


EvaluationResult(score=36.33, results=<list of 300 results>)

In [9]:
simba = dspy.SIMBA(metric=metric, max_steps=12, max_demos=10)
optimized_agent = simba.compile(agent, trainset=trainset, seed=6793115)

2025/07/20 13:04:44 INFO dspy.teleprompt.simba: Starting batch 1 of 12.
2025/07/20 13:05:17 INFO dspy.teleprompt.simba: Sampling program trajectories on 32 examples x 6 samples.


Processed 31 / 192 examples:  16%|█▌        | 31/192 [00:59<04:50,  1.80s/it]'composer_info_retriever'
'NoneType' object has no attribute 'answer'
Processed 63 / 192 examples:  33%|███▎      | 63/192 [01:44<01:56,  1.11it/s]'composer_info_retriever'
'NoneType' object has no attribute 'answer'
Processed 69 / 192 examples:  36%|███▌      | 69/192 [01:54<03:15,  1.59s/it]'general_person_lookup'
'NoneType' object has no attribute 'answer'
Processed 160 / 192 examples:  83%|████████▎ | 160/192 [04:00<01:05,  2.05s/it]'composer_info_retriever'
'NoneType' object has no attribute 'answer'
Processed 192 / 192 examples: 100%|██████████| 192/192 [04:41<00:00,  1.47s/it]

2025/07/20 13:09:58 INFO dspy.teleprompt.simba: Batch 1: Baseline mini-batch score: 0.2760416666666667

2025/07/20 13:09:58 INFO dspy.teleprompt.simba: Batch 1: Processing bucket #1, with max score 1.0, max-to-min gap 1.0, and max-to-avg gap 0.8333333333333334.


2025/07/20 13:10:02 INFO dspy.teleprompt.simba: Batch 1: Invoking strategy: append_a_rule
2025/07/20 13:10:11 INFO dspy.teleprompt.simba_utils: Advice for react.predict: When tasked with identifying familial relationships, ensure that the relationship type used is supported by the available functions. For genealogical queries, use valid relationship types such as 'father', 'mother', 'grandparents', etc., and avoid unsupported terms like 'grandfather' or 'ancestors'. If multiple attempts using valid relationship types fail, consider leveraging external historical knowledge or alternative methods to confirm details. Once the correct individual is identified, proceed with extracting and processing the required data (e.g., first name for vowel counting) accurately.
2025/07/20 13:10:11 INFO dspy.teleprompt.simba: 

2025/07/20 13:10:11 INFO dspy.teleprompt.simba: Batch 1: Processing bucket #2, with max score 1.0, max-to-min gap 1.0, and max-to-avg gap 0.8333333333333334.
2025/07/20 13:10:11 

  0%|          | 0/224 [00:00<?, ?it/s]'person_info_retriever'
'NoneType' object has no attribute 'answer'
Processed 61 / 224 examples:  27%|██▋       | 61/224 [01:27<04:25,  1.63s/it]'composer_info_retriever'
'NoneType' object has no attribute 'answer'
Processed 65 / 224 examples:  29%|██▉       | 65/224 [01:31<03:34,  1.35s/it]'enhanced_person_info_retriever'
'NoneType' object has no attribute 'answer'
Processed 70 / 224 examples:  31%|███▏      | 70/224 [01:35<02:35,  1.01s/it]'enhanced_director_mother_lookup'
'NoneType' object has no attribute 'answer'
Processed 96 / 224 examples:  43%|████▎     | 96/224 [02:11<02:15,  1.06s/it]'biographical_info_retriever'
'NoneType' object has no attribute 'answer'
Processed 100 / 224 examples:  45%|████▍     | 100/224 [02:16<02:20,  1.13s/it]'biographical_info_retriever'
'NoneType' object has no attribute 'answer'
Processed 127 / 224 examples:  57%|█████▋    | 127/224 [02:47<01:19,  1.22it/s]'composer_info_retriever'
'NoneType' object has no att

...

Processed 1 / 224 examples:   0%|          | 1/224 [00:03<13:45,  3.70s/it]'historical_figure_info_retriever'
'NoneType' object has no attribute 'answer'
Processed 2 / 224 examples:   1%|          | 2/224 [00:05<10:11,  2.76s/it]'date_calculator'
'NoneType' object has no attribute 'answer'
Processed 5 / 224 examples:   2%|▏         | 5/224 [00:09<05:56,  1.63s/it]'historical_figure_info_retriever'
'NoneType' object has no attribute 'answer'
Processed 12 / 224 examples:   5%|▌         | 12/224 [00:15<03:23,  1.04it/s]'genealogy_query'
'NoneType' object has no attribute 'answer'
Processed 32 / 224 examples:  14%|█▍        | 31/224 [00:39<04:00,  1.25s/it]'biographical_info_retriever'
'NoneType' object has no attribute 'answer'
Processed 36 / 224 examples:  16%|█▌        | 36/224 [00:46<04:27,  1.43s/it]'biographical_info_retriever'
'NoneType' object has no attribute 'answer'
Processed 45 / 224 examples:  20%|██        | 45/224 [00:54<02:45,  1.08it/s]'biographical_info_retriever'
'NoneTy

2025/07/20 15:56:00 INFO dspy.teleprompt.simba: Scores after 12 batches: [0.5, 0.46875, 0.46875, 0.46875, 0.53125, 0.5, 0.46875], Best: 0.53125



2025/07/20 15:56:01 INFO dspy.teleprompt.simba: VALIDATION: Evaluating 7 programs on the full trainset.


Processed 58 / 700 examples:   8%|▊         | 58/700 [01:04<10:43,  1.00s/it]'{ "historical_relationship_finder": {"function_name": "historical_relationship_finder", "arguments": "(figure_name: str, relationship_type: str, context: str = \'\', output_format: str = \'text\', date_range: dict = None, include_extended_family: bool = False)", "docstring": "No docstring."} }'
'NoneType' object has no attribute 'answer'
Processed 108 / 700 examples:  15%|█▌        | 108/700 [01:46<04:19,  2.28it/s]'date_calculator'
'NoneType' object has no attribute 'answer'
Processed 110 / 700 examples:  16%|█▌        | 110/700 [01:52<12:11,  1.24s/it]'historical_figure_info_retriever'
'NoneType' object has no attribute 'answer'
Processed 129 / 700 examples:  18%|█▊        | 128/700 [02:08<13:34,  1.42s/it]'date_calculator'
'NoneType' object has no attribute 'answer'
Processed 140 / 700 examples:  20%|██        | 140/700 [02:15<07:21,  1.27it/s]'historical_figure_info_retriever'
'NoneType' object has no att

2025/07/20 16:06:09 INFO dspy.teleprompt.simba: Final trainset scores: [0.22, 0.35, 0.43, 0.44, 0.39, 0.47, 0.5], Best: 0.5 (at index 6)





In [10]:
evaluate(optimized_agent)

Average Metric: 25.00 / 45 (55.6%):  15%|█▌        | 45/300 [00:25<02:22,  1.79it/s]

2025/07/20 16:06:39 ERROR dspy.utils.parallelizer: Error for Example({'question': "What is the date 6 days after when Prince Nikolai Of Denmark's mother was born?", 'answer': '1964-07-06', 'functions': {'royal_family_info_retriever': <function royal_family_info_retriever at 0x13d52d4e0>, 'sports_figure_bio_retriever': <function sports_figure_bio_retriever at 0x13d52d580>, 'date_calculator': <function date_calculator at 0x13d52d6c0>, 'finish': <function finish at 0x13d2ea520>}}) (input_keys={'functions', 'question'}): 'biographical_info_retriever'. Set `provide_traceback=True` for traceback.


Average Metric: 28.00 / 49 (57.1%):  17%|█▋        | 50/300 [00:26<01:10,  3.54it/s]

2025/07/20 16:06:40 ERROR dspy.utils.parallelizer: Error for Example({'question': "What is the date 7 days after the director of film Kalteva Torni 's birthday?", 'answer': '1963-11-07', 'functions': {'film_info_retriever': <function film_info_retriever at 0x13d52de40>, 'sports_figure_info_retriever': <function sports_figure_info_retriever at 0x13d52dee0>, 'date_calculator': <function date_calculator at 0x13d52e0c0>, 'finish': <function finish at 0x13d2ea520>}}) (input_keys={'functions', 'question'}): 'biographical_info_retriever'. Set `provide_traceback=True` for traceback.


Average Metric: 50.00 / 82 (61.0%):  28%|██▊       | 83/300 [00:39<00:56,  3.86it/s]

...

Average Metric: 152.00 / 257 (59.1%):  93%|█████████▎| 278/300 [02:23<00:09,  2.35it/s]

2025/07/20 16:08:37 ERROR dspy.utils.parallelizer: Error for Example({'question': "What is the sum of all the digits in the day, month and year of when Prince Nikolai Of Denmark's mother was born?", 'answer': '29', 'functions': {'family_relationship_finder': <function family_relationship_finder at 0x13e0459e0>, 'biographical_info_retriever': <function biographical_info_retriever at 0x13e045b20>, 'date_digit_sum': <function date_digit_sum at 0x13e045a80>, 'finish': <function finish at 0x13d2ea520>}}) (input_keys={'functions', 'question'}): litellm.RateLimitError: RateLimitError: OpenAIException - Rate limit reached for gpt-4o in organization org-x03WTFMofvP4w8cBY4XBkePo on tokens per min (TPM): Limit 2000000, Used 1994852, Requested 6965. Please try again in 54ms. Visit https://platform.openai.com/account/rate-limits to learn more.. Set `provide_traceback=True` for traceback.


Average Metric: 166.00 / 278 (59.7%): 100%|██████████| 300/300 [02:35<00:00,  1.92it/s]

2025/07/20 16:08:49 INFO dspy.evaluate.evaluate: Average Metric: 166.0 / 300 (55.3%)


EvaluationResult(score=55.33, results=<list of 300 results>)

In [11]:
dspy.inspect_history()





[2025-07-20T16:08:49.545101]

System message:

Your input fields are:
1. `question` (str): 
2. `trajectory` (str): 
3. `functions` (str):
Your output fields are:
1. `reasoning` (str): 
2. `next_selected_fn` (str): 
3. `args` (dict[str, Any]):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## trajectory ## ]]
{trajectory}

[[ ## functions ## ]]
{functions}

[[ ## reasoning ## ]]
{reasoning}

[[ ## next_selected_fn ## ]]
{next_selected_fn}

[[ ## args ## ]]
{args}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "additionalProperties": true}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        For the final answer, produce short (not full sentence) answers in which you format dates as YYYY-MM-DD, names as Firstname Lastname, and numbers without leading 0s.
        
        If the module receives a question involving identifying a p

In [12]:
optimized_agent.save("optimized_gpt4o.json", save_program=False)